<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_6/blob/main/Session6_Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sys
import pandas as pd
import numpy as np
import csv

!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 1.0MB 13.2MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=9576724077785e5428201ece38b4d68744f4b9d0c1e7972a12ac86e24f997964
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [4]:
import random
import googletrans
from googletrans import Translator

def translate(sentence_translated):
  # translator = Translator(service_urls=['translate.googleapis.com'])
  translator = Translator()
  # sentence = ['']

  # available_langs = list(googletrans.LANGUAGES.keys()) 
  # trans_lang = random.choice(available_langs) 
  # print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

  sentence_translated = pd.DataFrame(sentence_translated.iloc[0:200,:]).reset_index().drop(columns=['index'])

  trans_array = np.array(sentence_translated['tweets'])
  # int(len(trans_array)/100)

  for i in range(200):
    translations = translator.translate(trans_array[i],src='en', dest='czech') 
    #print(translations)
    t_text = translations.text
    print('czech: ',t_text)

    translations_en_random = translator.translate(t_text, src='czech', dest='en') 
    en_text = translations_en_random.text
    print('Eng: ',en_text)
    trans_array[i] = en_text


  sentence_translated['tweets'] = trans_array
  return sentence_translated



In [5]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [6]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [48]:
import torch.nn as nn
import torch.nn.functional as F

class encoders(nn.Module):

  def __init__(self,vocab_size, embedding_dim, hidden_dim, n_layers, dropout):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim)
    self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
    
  def forward(self,text,text_lengths):
    embedded = self.embedding(text)
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
    packed_output, (hidden, cell) = self.encoder(packed_embedded)
    return packed_output, hidden

class decoders(nn.Module):

  def __init__(self, input_to_decoder_size, decoder_hidden_size, no_times_decoder_cell_has_to_run):
    super().__init__()
    self.decoder_single_rnn_cell = nn.LSTMCell(input_to_decoder_size,decoder_hidden_size)
    self.no_times_decoder_cell_has_to_run = no_times_decoder_cell_has_to_run
    self.decoder_hidden_size = decoder_hidden_size

  def forward(self, encoder_context_vector):
    encoder_context_vector = encoder_context_vector.squeeze(0)
    hx = torch.zeros(encoder_context_vector.size(0),self.decoder_hidden_size).to(device)
    cx = torch.zeros(encoder_context_vector.size(0),self.decoder_hidden_size).to(device)
    otpt = []
    for i in range(self.no_times_decoder_cell_has_to_run):
      hx,cx = self.decoder_single_rnn_cell(encoder_context_vector,(hx,cx))
      otpt.append(hx)
      # print(i,hx.shape)
    otpt = torch.stack(otpt,dim = 0)
    return otpt, hx

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, encoder, decoder, hidden_dim, output_dim):
        
      super().__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self,src,src_len):
      # print('combined')
      # print(src.shape)
      enc_packed_outputs, enc_hidden = self.encoder(src,src_len)
      # print(enc_hidden.shape)
      dec_otpt, dec_hidden = self.decoder(enc_hidden)
      # print(dec_hidden.shape)
      dense_outputs = self.fc(dec_hidden)
      #print(dense_outputs.shape)
      op = F.softmax(dense_outputs, dim =1)
      # return dense_outputs
      return op

In [8]:
tweet_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tweets (1).csv")
tweet_data.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [9]:
tweet_data.shape

(1364, 2)

In [10]:
tweet_data.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(tweet_data, test_size=0.3)
# (train, valid) = tweet_data.split(split_ratio=[85, 15], random_state = random.seed(SEED))
train = train.reset_index().drop(columns=['index'])
valid = valid.reset_index().drop(columns=['index'])

In [12]:
def delete(sent_delete):
  del_array = np.array(sent_delete['tweets'])
  for i in range(len(del_array)):
    words = str(del_array[i]).split()
    word_selected = random.choice(words)
    word_deleted = random_deletion(word_selected, 0.4)
    del_word = ''
    for k in word_deleted:
      del_word+=k
    del_array[i] = str(del_array[i]).replace(word_selected,del_word)
  sent_delete['tweets'] = del_array
  return sent_delete

tweet_delete = pd.DataFrame(train.iloc[:,:]).reset_index().drop(columns=['index'])
print(tweet_delete.head())
tweet_delete = delete(tweet_delete)

                                              tweets  labels
0  RT @markknoller: Obama Campaign says Romney sh...       1
1  Top Secret Obama 2012 World War 3 Illuminati A...       0
2  RT @JeffersonObama: Arianna Huffington blastin...       1
3  @ExtremeLiberal obama is the best choice by fa...       0
4  @AnitaMarks @barackobama @mittromney really? I...       0


In [13]:
print(tweet_delete.head(10))

                                              tweets  labels
0  RT @markknoller: Obama Campaign says Romney sh...       1
1  Top Secret Obama 2012 World War 3 Illuminati A...       0
2  RT @JeffersonObama: Arianna Huffington blastin...       1
3  @ExtremeLiberal obama is the best choice by fa...       0
4  @AnitaMarks @barackobama @mittromney really? I...       0
5  Harry "Yes, I'm secretly Barack Obama." Me" An...       0
6  RT @LouTommoBum: Michelle Obama invited the by...       0
7  Obama Concedes That Courts Can Review Acts of ...       0
8  If Obama win 2012 Election wait til 2016 he wi...       2
9  #WhatsRomneyHiding That time he told AIPAC Jer...       0


In [14]:
tweet_swap = pd.DataFrame(train.iloc[:,:]).reset_index().drop(columns=['index']) 
print(tweet_swap.head())

                                              tweets  labels
0  RT @markknoller: Obama Campaign says Romney sh...       1
1  Top Secret Obama 2012 World War 3 Illuminati A...       0
2  RT @JeffersonObama: Arianna Huffington blastin...       1
3  @ExtremeLiberal obama is the best choice by fa...       0
4  @AnitaMarks @barackobama @mittromney really? I...       0


In [15]:
def swap(tweet_swap):
  for i in tweet_swap.index:
    words = tweet_swap['tweets'][i].split()
    if len(words) <=1:
      continue
    swapped_array = random_swap(words)
    tweet_swap['tweets'][i] = ' '.join([str(elem) for elem in swapped_array])
  return tweet_swap

In [16]:
tweet_swap = swap(tweet_swap)
tweet_swap.head(6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,tweets,labels
0,RT @markknoller: Obama Campaign says make of r...,1
1,Secret Top 2012 3 World Antichrist War Illumin...,0
2,RT any Arianna President explains http://t.co/...,1
3,by i is check best choice @ExtremeLiberal far ...,0
4,@AnitaMarks @barackobama @mittromney rather I'...,0
5,"Harry Obama."" secretly ""Yes, Barack secretly M...",0


In [17]:
print(train.shape)
print(tweet_delete.shape)
print(tweet_swap.shape)

(954, 2)
(954, 2)
(954, 2)


In [18]:
tweet_translated = train
tweet_translated = translate(tweet_translated)
print(tweet_translated.shape)

czech:  RT @ markknoller: Obamova kampaň říká, že Romney by měl zveřejnit 23 let daňových přiznání, aby Američané mohli posoudit jakýkoli střet zájmů.
Eng:  RT @markknoller: Obama's campaign says Romney should release 23 years of tax returns so Americans can assess any conflict of interest.
czech:  Přísně tajné Obama 2012, 3. světová válka, iluminátské antikristovo spiknutí !: http://t.co/iqg1xarL přes @youtube
Eng:  Top Secret Obama 2012, World War 3, Illuminati Antichrist Conspiracy !: http://t.co/iqg1xarL via @youtube
czech:  RT @JeffersonObama: Arianna Huffingtonová při jakékoli příležitosti odstřelila prezidenta Obamu a chválila GOP. Tento obrázek vysvětluje věci. http://t.co/Ov6I5CoS
Eng:  RT @JeffersonObama: Arianna Huffington blasted President Obama at every opportunity and praised the GOP. This picture explains things. http://t.co/Ov6I5CoS
czech:  @ExtremeLiberal obama je zdaleka nejlepší volbou pro rok 2012. Tlačím lidi, aby hodně volili. zkontrolujte tweety z roku 2010. ale 

In [19]:
tweet_data_1 = pd.concat([train,tweet_swap,tweet_delete,tweet_translated]).reset_index().drop(columns=['index'])
tweet_data_1.head()

,tweets,labels
0,RT @markknoller: Obama Campaign says Romney sh...,1
1,Top Secret Obama 2012 World War 3 Illuminati A...,0
2,RT @JeffersonObama: Arianna Huffington blastin...,1
3,@ExtremeLiberal obama is the best choice by fa...,0
4,@AnitaMarks @barackobama @mittromney really? I...,0


In [20]:
tweet_data_1.labels.value_counts()

0    2086
1     788
2     188
Name: labels, dtype: int64

In [21]:
new_tweet_delete_1 = tweet_data_1[tweet_data_1.labels==1].reset_index().drop(columns=['index'])
new_tweet_delete_2 = tweet_data_1[tweet_data_1.labels==2].reset_index().drop(columns=['index'])
new_tweet_delete = pd.concat([new_tweet_delete_1,new_tweet_delete_2]).reset_index().drop(columns=['index'])
new_tweet_delete.head()

,tweets,labels
0,RT @markknoller: Obama Campaign says Romney sh...,1
1,RT @JeffersonObama: Arianna Huffington blastin...,1
2,#whatsObamahiding Kryptonite for the Grand Old...,1
3,RT @JeffersonObama: 62% of U.S. Jews would ree...,1
4,"Saul says Pres Obama ""will do anything"" to dis...",1


In [22]:
new_tweet_delete = delete(new_tweet_delete)
new_tweet_delete = swap(new_tweet_delete)
new_tweet_delete.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,tweets,labels
0,RT @markknoller: public assess years Romney sh...,1
1,RT @JeffersonObama: Arianna Huffington explain...,1
2,2012 Kryptonite for Old Grand Party! Obama #wh...,1
3,"RT Obama, would of U.S. Jews 4% reelect @Jeffe...",1
4,"Saul says Obama from ""will do anything"" to dis...",1


In [23]:
tweet_data = pd.concat([tweet_data_1,new_tweet_delete]).reset_index().drop(columns=['index'])
print(tweet_data.head())
print(tweet_data.shape)
print(tweet_data.labels.value_counts())

                                              tweets  labels
0  RT @markknoller: Obama Campaign says Romney sh...       1
1  Top Secret Obama 2012 World War 3 Illuminati A...       0
2  RT @JeffersonObama: Arianna Huffington blastin...       1
3  @ExtremeLiberal obama is the best choice by fa...       0
4  @AnitaMarks @barackobama @mittromney really? I...       0
(4038, 2)
0    2086
1    1576
2     376
Name: labels, dtype: int64


In [24]:
import random
import torch, torchtext
from torchtext import data
SEED = 43
torch.manual_seed(SEED)

In [25]:
Tweet = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [26]:
fields = [('tweet', Tweet), ('label', Label)]

In [27]:
example = [torchtext.legacy.data.Example.fromlist([tweet_data.tweets[i],tweet_data.labels[i]], fields) for i in range(tweet_data.shape[0])] 

In [28]:
twitterDataset = torchtext.legacy.data.Dataset(example, fields)

In [29]:
train_data = twitterDataset

In [30]:
print(valid.shape)
print(valid.head())

(410, 2)
                                              tweets  labels
0  RT @Our4thEstate: @edshow The President of the...       1
1  RT @ohgirlphrase: American kid "You're from th...       0
2  Yes #WHFail RT @jltho: This #WhatsRomneyHiding...       1
3  @PAC43 Thats the problem...there is no clean e...       2
4  RT @ohgirlphrase: American kid "You're from th...       0


In [31]:
example_1 = [torchtext.legacy.data.Example.fromlist([valid.tweets[i],valid.labels[i]], fields) for i in range(valid.shape[0])] 

In [33]:
validDataset = torchtext.legacy.data.Dataset(example_1, fields)

In [34]:
valid = validDataset

In [35]:
len(train_data), len(valid)

(4038, 410)

In [36]:
vars(twitterDataset.examples[0])

{'label': 1,
 'tweet': ['RT',
  '@markknoller',
  ':',
  'Obama',
  'Campaign',
  'says',
  'Romney',
  'should',
  'make',
  'public',
  '23',
  'years',
  'of',
  'tax',
  'returns',
  'so',
  'Americans',
  'can',
  'assess',
  'any',
  'conflicts',
  'of',
  'interest',
  '.']}

In [37]:
Tweet.build_vocab(train_data)
Label.build_vocab(train_data)

In [38]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  5420
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 3522), ('.', 2780), ('#', 2683), (':', 2612), (',', 1989), ('the', 1827), ('RT', 1774), ('"', 1668), ('to', 1286), ('?', 1111)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train_data, valid), batch_size = 64, 
                                                            sort_key = lambda x: len(x.tweet),
                                                            sort_within_batch=True, device = device)

In [41]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

In [49]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 1
dropout = 0.4

# Instantiate the model
enc = encoders(size_of_vocab, embedding_dim, num_hidden_nodes, num_layers, dropout = dropout)
dec = decoders(num_hidden_nodes,num_hidden_nodes,3)

model = classifier(enc,dec,num_hidden_nodes,num_output_nodes).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [50]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (encoder): encoders(
    (embedding): Embedding(5420, 300)
    (encoder): LSTM(300, 100, batch_first=True, dropout=0.4)
  )
  (decoder): decoders(
    (decoder_single_rnn_cell): LSTMCell(100, 100)
  )
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 1,867,903 trainable parameters


In [51]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [45]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweet  
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweet
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [52]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.072 | Train Acc: 55.04%
	 Val. Loss: 1.019 |  Val. Acc: 73.88% 

	Train Loss: 0.996 | Train Acc: 59.99%
	 Val. Loss: 0.854 |  Val. Acc: 76.12% 

	Train Loss: 0.899 | Train Acc: 66.41%
	 Val. Loss: 0.815 |  Val. Acc: 73.44% 

	Train Loss: 0.799 | Train Acc: 75.93%
	 Val. Loss: 0.845 |  Val. Acc: 68.53% 

	Train Loss: 0.732 | Train Acc: 82.01%
	 Val. Loss: 0.823 |  Val. Acc: 70.54% 

	Train Loss: 0.691 | Train Acc: 86.01%
	 Val. Loss: 0.783 |  Val. Acc: 75.67% 

	Train Loss: 0.675 | Train Acc: 87.30%
	 Val. Loss: 0.825 |  Val. Acc: 71.43% 

	Train Loss: 0.656 | Train Acc: 89.06%
	 Val. Loss: 0.780 |  Val. Acc: 76.56% 

	Train Loss: 0.647 | Train Acc: 89.89%
	 Val. Loss: 0.785 |  Val. Acc: 76.34% 

	Train Loss: 0.643 | Train Acc: 90.38%
	 Val. Loss: 0.789 |  Val. Acc: 75.22% 



In [55]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    enc = encoders(size_of_vocab, embedding_dim, num_hidden_nodes, num_layers, dropout = dropout).to(device)
    encoder_packed_outputs, encoder_final_hidden = enc(tensor,length_tensor)
    print('Encoder LSTM output vector after each word: ')
    for i in range(encoder_packed_outputs.data.shape[0]):
      print('after',i+1, 'word')
      print(encoder_packed_outputs.data[i])
    print()
    print('Encoder LSTM final hidden vector')
    print(encoder_final_hidden)
    print()
    dec = decoders(num_hidden_nodes,num_hidden_nodes,3).to(device)
    decoder_outputs, decoder_final_hidden = dec(encoder_final_hidden)
    print('Decoder: ',decoder_outputs)
    print('Final vector after FC layer')
    print(prediction)
    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [57]:
classify_tweet("I didn't like it.")

Encoder LSTM output vector after each word: 
after 1 word
tensor([ 3.3666e-01, -1.1060e-01,  1.4279e-01,  1.9930e-02,  9.4484e-02,
         4.4644e-01, -2.4611e-01, -1.7161e-01, -2.7330e-01,  4.9464e-02,
         2.5922e-01,  5.5827e-02,  1.2597e-01,  4.6192e-02,  4.5015e-03,
        -8.1919e-02,  3.7158e-02, -3.1237e-02,  2.8741e-01,  6.7959e-02,
        -1.2523e-01, -4.4651e-01,  2.1111e-01,  1.5342e-01, -2.4255e-01,
         3.6058e-02,  2.7419e-02, -6.2437e-02,  5.6175e-02,  7.1488e-03,
        -2.9687e-02, -2.6871e-02,  5.2023e-02, -2.3182e-01,  7.6664e-02,
        -3.7827e-01,  4.4040e-02, -1.4775e-01,  1.1352e-01,  4.8270e-02,
         2.3656e-02, -1.4615e-01, -7.3957e-02,  9.2659e-02,  4.1937e-05,
         3.0000e-03,  2.4108e-04, -1.8857e-02, -3.3915e-01, -7.7533e-02,
         2.1053e-03,  2.4549e-03,  1.7717e-02,  1.0689e-01,  7.8248e-03,
        -1.4617e-01,  1.6179e-02, -4.7919e-02,  2.5926e-02, -1.0161e-01,
        -6.9049e-02,  3.6792e-01,  1.5034e-01, -5.1396e-02, -8.856

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


'Negative'

In [58]:
classify_tweet("I like it very much.")

Encoder LSTM output vector after each word: 
after 1 word
tensor([-1.9363e-02,  1.8065e-02, -2.8903e-01,  4.0063e-02, -2.0260e-01,
         3.2156e-01,  9.2181e-02, -1.5146e-01, -3.6137e-02,  1.5422e-01,
        -5.0807e-02, -1.9956e-01,  2.6497e-01,  1.5487e-02,  1.6894e-01,
         1.8305e-02,  1.0102e-01,  1.3011e-01, -1.0462e-01,  3.6302e-02,
         6.3942e-02,  7.4309e-02,  6.4892e-02,  9.1315e-02,  1.5883e-01,
         3.5116e-01, -2.5601e-01,  3.1382e-02, -2.2766e-01,  8.7207e-04,
        -5.7092e-02, -9.9095e-02,  2.8005e-01, -3.9773e-01, -1.5462e-01,
        -8.1478e-02, -1.6369e-01,  1.9323e-01,  7.7096e-02, -1.5990e-01,
         3.2599e-01,  2.4637e-01, -9.6007e-02,  6.7740e-02,  1.1701e-01,
         5.0809e-01, -1.0506e-01, -2.0883e-02, -1.6657e-01,  5.9626e-01,
        -1.0922e-01, -7.0369e-03,  2.6509e-02, -2.1007e-02, -1.2084e-01,
        -4.1167e-01, -4.7511e-01,  2.8953e-02, -2.7167e-01, -9.7713e-02,
         6.2128e-03, -2.2218e-02,  3.3583e-02,  2.4181e-01,  5.144

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


'Positive'